# Build quarto file for api reference
The created `api.qmd` is cleanned with a ' #' replace '#'  then moved to Documentation folder.

In [ ]:
from BI.Data.manip import manip
from BI.Utils.Gaussian import gaussian
from BI.Utils.Effects import effects 
from BI.SetDevice.set import setup_device

from BI.Utils.link import link
from BI.Diagnostic.Diag import diag
from BI.Network.Net import net
from BI.NBDA.NBDA import NBDA
from BI.Models.surv import survival
from BI.Models.GMM import *
from BI.Models.DPMM import *
from BI.Models.models import models
from BI.ML.ml import ml
from BI.BNN.bnn import bnn 
from numpyro.infer import MCMC, NUTS, Predictive
from numpyro.handlers import condition


In [3]:
import os
import re
import inspect
import textwrap
from BI import bi
from typing import Callable

def get_function_signature(func: Callable) -> str:
    """
    Inspects a function and returns its full signature as a formatted string,
    with each argument on a new, indented line.

    Args:
        func (Callable): The function object to inspect.

    Returns:
        A string containing the formatted python markdown block.
    """
    try:
        name = func.__name__
        sig = inspect.signature(func)
        params = sig.parameters.values()
        
        # Handle the simple case of a function with no parameters
        if not params:
            signature_string = f"bi.dist.{name}()"
        else:
            indent = "    "  # 4 spaces for indentation
            
            # Create a list of string representations for each parameter
            param_strings = [str(p) for p in params]
            
            # Join the parameters with a comma, newline, and the indent
            args_joined = f",\n{indent}".join(param_strings)
            
            # Assemble the final multi-line signature string
            # Includes a recommended trailing comma on the last argument
            signature_string = f"bi.dist.{name}(\n{indent}{args_joined},\n)"

        # Wrap the result in the markdown code block
        return f"```python\n{signature_string}\n```"
        
    except (TypeError, ValueError) as e:
        # Handle cases where the input is not a valid inspectable object
        error_message = f"Could not get the signature for '{func}': {e}"
        return f"```text\n{error_message}\n```"

def split_docstring_after_args(docstring) :
    """
    Splits a docstring into two parts, with the split occurring immediately
    after the '#### Args:' delimiter.

    The delimiter itself is included at the end of the first part.

    Args:
        docstring (str): The input docstring to be split.

    Returns:
        A tuple containing two strings:
        - The first string is the content *up to and including* '#### Args:'.
        - The second string is the content *after* '#### Args:'.
          If the delimiter is not found, the second string will be empty.
    """
    # Clean up the docstring's indentation first
    dedented_docstring = textwrap.dedent(docstring).strip()
    
    delimiter = '#### Args:'
    
    # Find the starting position of the delimiter
    delimiter_index = dedented_docstring.find(delimiter)
    
    # If the delimiter is found in the string
    if delimiter_index != -1:
        # Calculate the position right after the delimiter
        split_point = delimiter_index + len(delimiter)
        
        # The first part is everything up to and including the delimiter
        part_before = dedented_docstring[:split_point].strip()
        
        # The second part is everything after the delimiter
        part_after = dedented_docstring[split_point:].strip()
        
        return part_before, part_after
    else:
        # If the delimiter is not found, return the whole docstring as the
        # first part and an empty string for the second.
        return dedented_docstring, ""

def wrap_example_in_place(docstring: str) -> str:
    """
    Finds the '#### Example Usage:' section in a docstring and replaces its
    content with a version wrapped in a python markdown code block.

    Args:
        docstring (str): The input docstring to modify.

    Returns:
        The entire docstring with the 'Example Usage' section modified.
        If the section is not found, the original docstring is returned.
    """
    # Clean up common indentation from the entire docstring
    dedented_docstring = textwrap.dedent(docstring)
    dedented_docstring = '\n' + dedented_docstring

    # Compile the regex for clarity and efficiency.
    # It captures the content of the 'Example Usage' section.
    pattern = re.compile(
        r"(#### Example Usage:)(.*?)(?=####|$)",
        re.DOTALL
    )

    # Define a "replacer" function that will be called for each match.
    # It takes a match object as its argument.
    def replacer(match):
        # group(1) is the "#### Example Usage:" tag itself.
        # group(2) is the code content we captured.
        tag = match.group(1)
        code_content = match.group(2)
        cleaned_code = textwrap.dedent(code_content).strip()
        
        # Build the new replacement string with the wrapped code
        return f"{tag}\n```python\n{cleaned_code}\n```\n\n"

    # Use re.sub() to find the pattern and replace it using our replacer function.
    # The `count=1` argument ensures it only replaces the first occurrence.
    modified_docstring = pattern.sub(replacer, dedented_docstring, count=1)
    
    return modified_docstring

def remove_all_leading_whitespace(text: str) -> str:
    """
    Aggressively removes all leading whitespace (spaces, tabs, etc.) from
    every line in a given multi-line string.

    This is the most robust method for ensuring every line starts at column 1,
    especially when dealing with mixed tabs and spaces.

    Args:
        text (str): The input string to clean.

    Returns:
        A cleaned string where every line is left-aligned.
    """
    # 1. Split the entire string into a list of individual lines.
    lines = text.splitlines()
    
    # 2. Create a new list where .lstrip() has been applied to each line.
    #    .lstrip() removes all kinds of leading whitespace by default.
    stripped_lines = [line.lstrip() for line in lines]
    
    # 3. Join the cleaned lines back together into a single string.
    return '\n'.join(stripped_lines)

def math_to_dolar(input_string):
    pattern = r':num:math:`(.*?)`'
    replacement = r'$\1$'
    return re.sub(pattern, replacement, input_string)

def math_to_dolar2(input_string):
    pattern = r':math:`(.*?)`'
    replacement = r'$\1$'
    return re.sub(pattern, replacement, input_string)

def get_docstring(func):   
    docstring = inspect.getdoc(func)
    docstring = math_to_dolar(docstring)
    docstring = math_to_dolar2(docstring)
    before, after = split_docstring_after_args(docstring)
    call = get_function_signature(func)
    after = wrap_example_in_place(after)

    warp = before  + "\n\n" + call + "\n\n"  + after + "\n\n" + '---\n'
    warp =  remove_all_leading_whitespace(warp)
    lines = warp.splitlines()
    processed_lines = [
        line.lstrip() if line.lstrip().startswith('#') else line
        for line in lines
    ]
    return warp

def write_to_file(filename: str, content: str, append: bool = False):
    """
    Writes content to a specified file, with an option to append.

    This function is ideal for creating or updating text-based files like
    QMD, MD, or TXT documents.

    Args:
        filename (str): The path and name of the file to write to.
        content (str): The string content to write to the file.
        append (bool): 
            - If False (default), the file will be overwritten with the new 
              content.
            - If True, the new content will be added to the end of the file.
              If the file does not exist, it will be created.
    """
    # Determine the mode: 'w' for write (overwrite), 'a' for append.
    mode = 'a' if append else 'w'
    
    try:
        # Use 'with open' for safe file handling (ensures file is closed).
        # encoding='utf-8' is a best practice for text files.
        with open(filename, mode, encoding='utf-8') as f:
            f.write(content)
        
        operation = "Appended to" if append else "Successfully wrote to"
        print(f"{operation} '{filename}'")

    except IOError as e:
        print(f"Error: Could not write to file '{filename}'. Reason: {e}")


## Dist class

In [4]:
m= bi(platform='cpu')
non_dunder_callables = [
    attr for attr in dir(m.dist)
    if callable(getattr(m.dist, attr)) and not attr.startswith("__")
]
i = 0
for name in non_dunder_callables:
    callable_obj = getattr(m.dist, name)
    underlying_func = inspect.unwrap(callable_obj)
    docstring = underlying_func.__doc__
    if docstring is None:
        pass
    else:        
        if i == 0:
            string1 = """---\ntitle: "Distributions"\nsidebar: api\n---\n\n"""
            string2 = """ \n `Utils.np_dists.UnifiedDist` is a class to unify various distribution methods and provide a     consistent interface for sampling and inference.\n\n"""
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_dist.qmd', content = string1 + string2 + warp + "\n\n",  append = False)
            i = i + 1

        else:
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_dist.qmd', content =  warp + "\n\n",  append = True)
            i = i + 1




jax.local_device_count 16
Successfully wrote to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_dist.qmd'
Appended to 'api_di

## Diag Class

In [1]:
non_dunder_callables = [
    attr for attr in dir(diag)
    if callable(getattr(diag, attr)) and not attr.startswith("__")
]
i = 0
for name in non_dunder_callables:
    callable_obj = getattr(diag, name)
    underlying_func = inspect.unwrap(callable_obj)
    docstring = underlying_func.__doc__
    if docstring is None:
        pass
    else:        
        if i == 0:
            string1 = """---\ntitle: "Diagnostics"\nsidebar: api\n---\n\n"""
            string2 = """\n `diag.diag` is a class to unify various diagnostics methods and provide a consistent interface for diagnostics.\n\n"""
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_diag.qmd', content = string1 + string2 + warp + "\n\n",  append = False)
            i = i + 1

        else:
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_diag.qmd', content =  warp + "\n\n",  append = True)
            i = i + 1



NameError: name 'diag' is not defined

## Manip

In [ ]:
non_dunder_callables = [
    attr for attr in dir(manip)
    if callable(getattr(manip, attr)) and not attr.startswith("__")
]
i = 0
for name in non_dunder_callables:
    callable_obj = getattr(manip, name)
    underlying_func = inspect.unwrap(callable_obj)
    docstring = underlying_func.__doc__
    if docstring is None:
        pass
    else:        
        if i == 0:
            string1 = """---\ntitle: "Manipulate data"\nsidebar: api\n---\n\n"""
            string2 = """\n `manip` is a class to unify various diagnostics methods and provide a consistent interface for diagnostics.\n\n"""
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_manip.qmd', content = string1 + string2 + warp + "\n\n",  append = False)
            i = i + 1

        else:
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_manip.qmd', content =  warp + "\n\n",  append = True)
            i = i + 1


Successfully wrote to 'api_manip.qmd'
Appended to 'api_manip.qmd'
Appended to 'api_manip.qmd'
Appended to 'api_manip.qmd'
Appended to 'api_manip.qmd'
Appended to 'api_manip.qmd'
Appended to 'api_manip.qmd'
Appended to 'api_manip.qmd'
Appended to 'api_manip.qmd'


## Setup

In [ ]:
setup_device.__doc__

"## Configures JAX for distributed computation.\n\n    This function sets up the JAX computing environment by specifying the hardware \n    platform and managing CPU core allocation. It also handles deallocation of existing\n    devices and configures XLA flags appropriately.\n\n    ### Args:\n    \n    - *platform:* str, optional\n        The hardware platform to use for computation. Options include:\n        - 'cpu': Use CPU(s) for computation\n        - 'gpu': Use GPU(s) for computation\n        - 'tpu': Use TPU(s) for computation\n        Defaults to 'cpu'.\n\n    - *cores:* int, optional\n        Number of CPU cores to allocate for computation. If None, all available CPU \n        cores will be used. Only applicable when platform is 'cpu'.\n        \n    - *deallocate:* bool, optional\n        Whether to deallocate any existing devices before setting up new configuration.\n        Defaults to False.\n\n    ### Notes\n    This function must be called before any JAX imports or usage

In [ ]:
non_dunder_callables = [
    attr for attr in dir(setup_device)
    if callable(getattr(setup_device, attr)) and not attr.startswith("__")
]
i = 0
for name in non_dunder_callables:
    callable_obj = getattr(setup_device, name)
    underlying_func = inspect.unwrap(callable_obj)
    docstring = underlying_func.__doc__
    if docstring is None:
        pass
    else:        
        if i == 0:
            string1 = """---\ntitle: "Setup"\nsidebar: api\n---\n\n"""
            string2 = """\n `setup_device` is a class to parametrize JAX computing environment.\n\n"""
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_setup.qmd', content = string1 + string2 + warp + "\n\n",  append = False)
            i = i + 1

        else:
            warp = get_docstring(underlying_func)
            write_to_file(filename = 'api_setup.qmd', content =  warp + "\n\n",  append = True)
            i = i + 1

